In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from functools import partial
from scipy.integrate import odeint
from sympy import symbols, Eq, solve, Function, Matrix, diff

# Define the ODE systems
def fast_system(y, t, eps):
    phi, u, c1, c2, j1, j2, w = y
    dphidt = u 
    dudt   = -z1 * c1 - z2 * c2
    dc1dt  = -z1 * c1 * u - eps * j1
    dc2dt  = -z2 * c2 * u - eps * j2
    dj1dt  = 0
    dj2dt  = 0
    dwdt   = eps
    
    return [dphidt, dudt, dc1dt, dc2dt, dj1dt, dj2dt, dwdt]


def subfast_system(y, t, eps):
    phi, u, c1, c2, j1, j2, w = y
    dphidt = u 
    dudt   = -z1 * c1 - z2 * c2
    dc1dt  = -z1 * c1 * u 
    dc2dt  = -z2 * c2 * u 
    dj1dt  = 0
    dj2dt  = 0
    dwdt   = 0
    
    return [dphidt, dudt, dc1dt, dc2dt, dj1dt, dj2dt, dwdt]



t_end = 20
t   =  np.linspace(0, t_end, 100)
t2  =  np.linspace(0,-t_end, 100)
eps = 0.01
T_slow_end = 1
tau = np.linspace(0, T_slow_end, 100)

t_tensor = torch.tensor(t.reshape(-1, 1), dtype=torch.float64) 
tau_tensor = torch.tensor(tau.reshape(-1, 1), dtype=torch.float64)
t2_tensor = torch.tensor(t2.reshape(-1, 1), dtype=torch.float64)


def input_transform(t_tensor):
    return torch.cat([t_tensor], dim=1)

num_nrn = 7

class fast_system_PINN(nn.Module):
    def __init__(self):
        super(fast_system_PINN, self).__init__()
        self.fc1 = nn.Linear(1, num_nrn)
        self.fc2 = nn.Linear(num_nrn, num_nrn)
        self.fc3 = nn.Linear(num_nrn, 7)

    def forward(self, t):
        x = F.tanh(self.fc1(t))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

class slow_system_PINN(nn.Module):
    def __init__(self):
        super(slow_system_PINN, self).__init__()
        self.fc1 = nn.Linear(1, num_nrn)
        self.fc2 = nn.Linear(num_nrn, num_nrn)
        self.fc3 = nn.Linear(num_nrn, 7)

    def forward(self, tau):
        x = F.tanh(self.fc1(tau))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x
      
class fast_system_PINN2(nn.Module):
    def __init__(self):
        super(fast_system_PINN2, self).__init__()
        self.fc1 = nn.Linear(1, num_nrn)
        self.fc2 = nn.Linear(num_nrn, num_nrn)
        self.fc3 = nn.Linear(num_nrn, 7)

    def forward(self, t2):
        x = F.tanh(self.fc1(t2))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [2]:
z1, z2 = 1.0 , -1.0
V  = -10
l  = 1.0
l1 = l +  0.1
l2 = l -  0.1


r  = 0.5
r1 =  r -  0.1 
r2 =  r +  0.1


In [3]:
# Initial points:
c1_init, c2_init,  w_init =l1 , l2, 0.0  
print('The initial points of the BVP:', f"c1_init  = {c1_init}, c2_init = {c2_init}, w_init   = {w_init} ")

phi_slow_init, phi_slow_end = V, 0.0
#c1_slow_init, c1_slow_end   = l, r 
#c2_slow_init, c2_slow_end  = l, r
w_slow_init, w_slow_end  = 0.0, 1.0
u_a, u_b =  0.0, 0.0

c1_end, c2_end, w_end =  r1, r2 , 1.0
print('The ending points of the BVP:', f"c1_end  = {c1_end}, c2_end = {c2_end}, w_end   = {w_end} ")
 
#J1 = np.random.uniform(2 * min(0, V), 2 * max(0, V))/ abs(V)
# J1 is obtained as above because the sign of J_k is the same as z_kV+ ln(l_k/r_k)

#J2 = np.random.uniform(-max(0, V) * 2, -min(0, V) * 2)/ abs(V)


The initial points of the BVP: c1_init  = 1.1, c2_init = 0.9, w_init   = 0.0 
The ending points of the BVP: c1_end  = 0.4, c2_end = 0.6, w_end   = 1.0 


In [4]:
phys_weight  = 2
init_weight  = 1
bndry_weight = 1
def loss_func_fast(model, t_tensor, c1_init, c2_init, w_init,\
                                    phi_slow_init, c1_slow_init, c2_slow_init, w_slow_init,\
                                    eps, random_points=10):
    t_tensor.requires_grad = True
    pred_fast = model(t_tensor)
    phi_pred_fast, u_pred_fast = pred_fast[:, 0].unsqueeze(1), pred_fast[:, 1].unsqueeze(1)
    c1_pred_fast, c2_pred_fast = pred_fast[:, 2].unsqueeze(1), pred_fast[:, 3].unsqueeze(1)
    j1_pred_fast, j2_pred_fast = pred_fast[:, 4].unsqueeze(1), pred_fast[:, 5].unsqueeze(1)
    w_pred_fast = pred_fast[:, 6].unsqueeze(1)
    #ones = torch.ones_like(x_pred_fast, requires_grad=True)    
    dphi_dt = torch.autograd.grad(phi_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    du_dt = torch.autograd.grad(u_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    dc1_dt = torch.autograd.grad(c1_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    dc2_dt = torch.autograd.grad(c2_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    dj1_dt = torch.autograd.grad(j1_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    dj2_dt = torch.autograd.grad(j2_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    dw_dt = torch.autograd.grad(w_pred_fast.sum(), t_tensor, retain_graph=True, create_graph=True)[0]
    
    #for the fast system
    residual1_fast =  dphi_dt - u_pred_fast
    residual2_fast =  du_dt + z1 * c1_pred_fast + z2 * c2_pred_fast
    residual3_fast =  dc1_dt + z1 * c1_pred_fast * u_pred_fast + eps * j1_pred_fast
    residual4_fast =  dc2_dt + z2 * c2_pred_fast * u_pred_fast + eps * j2_pred_fast
    residual5_fast =  dj1_dt  
    residual6_fast =  dj2_dt 
    residual7_fast =  dw_dt - eps

    

    # for non-negativity of c1_pred_fast and c2_pred_fast
    residual_non_neg_c1f = torch.clamp(-c1_pred_fast, min=0)
    residual_non_neg_c2f = torch.clamp(-c2_pred_fast, min=0)
    
    init_loss_fast = torch.square(c1_pred_fast[0] - c1_init)  +\
                     torch.square(c2_pred_fast[0] - c2_init)  +\
                     torch.square(w_pred_fast[0] - w_init) +\
                     torch.square(j1_pred_fast[0] - J1) +\
                     torch.square(j2_pred_fast[0] - J2) 
    
    physics_loss_fast = torch.mean(residual1_fast**2 + residual2_fast**2 +\
                                   residual3_fast**2 + residual4_fast**2 +\
                                   residual5_fast**2 + residual6_fast**2 +\
                                   residual7_fast**2 +\
                                   residual_non_neg_c1f**2 + residual_non_neg_c2f**2)
    boundary_loss_fast = torch.square(phi_pred_fast[-1] - phi_slow_init) +\
                         torch.square(u_pred_fast[-1] - u_a) +\
                         torch.square(c1_pred_fast[-1] - c1_slow_init) +\
                         torch.square(c2_pred_fast[-1] - c2_slow_init) +\
                         torch.square(j1_pred_fast[-1] - j1_pred_fast[0])+\
                         torch.square(j2_pred_fast[-1] - j2_pred_fast[0])+\
                         torch.square(z1*c1_pred_fast[-1] + z2*c2_pred_fast[-1]) +\
                         torch.square(w_pred_fast[-1] - w_slow_init) 

        
    total_loss_fast   = phys_weight * physics_loss_fast +\
                        init_weight * init_loss_fast + bndry_weight * boundary_loss_fast
                        
    return total_loss_fast

def loss_func_slow(model, tau_tensor, eps, random_points=10):
    tau_tensor.requires_grad = True
    pred_slow = model(tau_tensor)
    phi_pred_slow, u_pred_slow = pred_slow[:, 0].unsqueeze(1), pred_slow[:, 1].unsqueeze(1)
    c1_pred_slow, c2_pred_slow = pred_slow[:, 2].unsqueeze(1), pred_slow[:, 3].unsqueeze(1)
    j1_pred_slow, j2_pred_slow = pred_slow[:, 4].unsqueeze(1), pred_slow[:, 5].unsqueeze(1)
    w_pred_slow = pred_slow[:, 6].unsqueeze(1)
    #ones = torch.ones_like(x_pred_fast, requires_grad=True)    
    dphi_dtau = torch.autograd.grad(phi_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    du_dtau = torch.autograd.grad(u_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    dc1_dtau = torch.autograd.grad(c1_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    dc2_dtau = torch.autograd.grad(c2_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    dj1_dtau = torch.autograd.grad(j1_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    dj2_dtau = torch.autograd.grad(j2_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    dw_dtau = torch.autograd.grad(w_pred_slow.sum(), tau_tensor, retain_graph=True, create_graph=True)[0]
    
    #for the fast system
    p = - (z1 * j1_pred_slow + z2 * j2_pred_slow)/(z1 * (z1 - z2) * c1_pred_slow)
    #p2 = (z1 * j1_pred_slow + z2 * j2_pred_slow) #/(z1 * (z1 - z2) * c1_pred_slow)
        
    residual1_slow = u_pred_slow #- eps * p
    residual2_slow = dphi_dtau - p
    residual3_slow = dc1_dtau + z1 * c1_pred_slow * p + j1_pred_slow #* (z1 - z2)*z1*c1_pred_slow
    residual4_slow = dc2_dtau + z2 * c2_pred_slow * p + j2_pred_slow #* (z1 - z2)*z1*c1_pred_slow
    residual5_slow = dj1_dtau
    residual6_slow = dj2_dtau 
    residual7_slow = dw_dtau - 1 # - (z1 - z2) * z1 * c1_pred_slow
    residual8_slow = z1 * c1_pred_slow + z2 * c2_pred_slow

    
    # for non-negativity of c1_pred_fast and c2_pred_fast
    residual_non_neg_c1s = torch.clamp(-c1_pred_slow, min=0)
    residual_non_neg_c2s = torch.clamp(-c2_pred_slow, min=0)
       
    
    init_loss_slow = torch.square(phi_pred_slow[0] - phi_slow_init) +\
                     torch.square(u_pred_slow[0] - u_a) +\
                     torch.square(c1_pred_slow[0] - c1_slow_init)  +\
                     torch.square(c2_pred_slow[0] - c2_slow_init)  +\
                     torch.square(w_pred_slow[0] - w_slow_init)  +\
                     torch.square(j1_pred_slow[0] - J1) +\
                     torch.square(j2_pred_slow[0] - J2) 
    
    
    physics_loss_slow = torch.mean(residual1_slow**2 + residual2_slow**2 +\
                                   residual3_slow**2 + residual4_slow**2 +\
                                   residual5_slow**2 + residual6_slow**2 +\
                                   residual7_slow**2 + residual8_slow**2+\
                                   residual_non_neg_c1s**2 + residual_non_neg_c2s**2)
    boundary_loss_slow =   torch.square(phi_pred_slow[-1] - phi_slow_end) +\
                           torch.square(u_pred_slow[-1] - u_b) +\
                           torch.square(c1_pred_slow[-1] - c1_slow_end)  +\
                           torch.square(c2_pred_slow[-1] - c2_slow_end)  +\
                           torch.square(j1_pred_slow[-1] - j1_pred_slow[0])+\
                           torch.square(j2_pred_slow[-1] - j2_pred_slow[0])+\
                           torch.square(w_pred_slow[-1] - w_slow_end) 
    
    
    total_loss_fast   = phys_weight * physics_loss_slow +\
                        init_weight * init_loss_slow + bndry_weight * boundary_loss_slow
    return total_loss_fast


def loss_func_fast2(model, t2_tensor, c1_end, c2_end, w_end,\
                                    phi_slow_end, c1_slow_end, c2_slow_end, w_slow_end,\
                                    eps, random_points=10):
    t2_tensor.requires_grad = True
    pred_fast2 = model(t2_tensor)
    phi_pred_fast2, u_pred_fast2 = pred_fast2[:, 0].unsqueeze(1), pred_fast2[:, 1].unsqueeze(1)
    c1_pred_fast2, c2_pred_fast2 = pred_fast2[:, 2].unsqueeze(1), pred_fast2[:, 3].unsqueeze(1)
    j1_pred_fast2, j2_pred_fast2 = pred_fast2[:, 4].unsqueeze(1), pred_fast2[:, 5].unsqueeze(1)
    w_pred_fast2 = pred_fast2[:, 6].unsqueeze(1)
    #ones = torch.ones_like(x_pred_fast, requires_grad=True)    
    dphi_dt2 = torch.autograd.grad(phi_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    du_dt2 = torch.autograd.grad(u_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    dc1_dt2 = torch.autograd.grad(c1_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    dc2_dt2 = torch.autograd.grad(c2_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    dj1_dt2 = torch.autograd.grad(j1_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    dj2_dt2 = torch.autograd.grad(j2_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    dw_dt2 = torch.autograd.grad(w_pred_fast2.sum(), t2_tensor, retain_graph=True, create_graph=True)[0]
    
    #for the fast system
    residual1_fast2 =  dphi_dt2 - u_pred_fast2
    residual2_fast2 =  du_dt2 + z1 * c1_pred_fast2 + z2 * c2_pred_fast2
    residual3_fast2 =  dc1_dt2 + z1 * c1_pred_fast2 * u_pred_fast2 + eps * j1_pred_fast2
    residual4_fast2 =  dc2_dt2 + z2 * c2_pred_fast2 * u_pred_fast2 + eps * j2_pred_fast2
    residual5_fast2 =  dj1_dt2  
    residual6_fast2 =  dj2_dt2 
    residual7_fast2 =  dw_dt2 - eps

    # for non-negativity of c1_pred_fast and c2_pred_fast
    residual_non_neg_c1f2 = torch.clamp(-c1_pred_fast2, min=0)
    residual_non_neg_c2f2 = torch.clamp(-c2_pred_fast2, min=0)

    
    init_loss_fast2 = torch.square(c1_pred_fast2[0] - c1_end)  +\
                      torch.square(c2_pred_fast2[0] - c2_end)  +\
                      torch.square(w_pred_fast2[0] - w_end) +\
                      torch.square(j1_pred_fast2[0] - J1) +\
                      torch.square(j2_pred_fast2[0] - J2) 
    
    physics_loss_fast2 = torch.mean(residual1_fast2**2 + residual2_fast2**2 +\
                                   residual3_fast2**2 + residual4_fast2**2 +\
                                   residual5_fast2**2 + residual6_fast2**2 +\
                                   residual7_fast2**2 +\
                                  residual_non_neg_c1f2**2 + residual_non_neg_c2f2**2)
    boundary_loss_fast2 = torch.square(phi_pred_fast2[-1] - phi_slow_end) +\
                          torch.square(u_pred_fast2[-1] - u_b) +\
                          torch.square(c1_pred_fast2[-1] - c1_slow_end) +\
                          torch.square(c2_pred_fast2[-1] - c2_slow_end) +\
                          torch.square(j1_pred_fast2[-1] - j1_pred_fast2[0])+\
                          torch.square(j2_pred_fast2[-1] - j2_pred_fast2[0])+\
                          torch.square(z1*c1_pred_fast2[-1] + z2*c2_pred_fast2[-1]) +\
                          torch.square(w_pred_fast2[-1] - w_slow_end) 
    
    total_loss_fast2   = phys_weight * physics_loss_fast2 +\
                         init_weight * init_loss_fast2 + bndry_weight * boundary_loss_fast2
    return total_loss_fast2


def total_loss_func(model_fast, model_slow, model_fast2, t_tensor, tau_tensor, t2_tensor,\
                    c1_init, c2_init, w_init,\
                    phi_slow_init, c1_slow_init, c2_slow_init, w_slow_init,\
                    c1_end, c2_end, w_end,\
                    phi_slow_end, c1_slow_end, c2_slow_end, w_slow_end,\
                    eps, weight_fast=1.0, weight_slow=1.0):
    loss_fast = loss_func_fast(model_fast, t_tensor, c1_init, c2_init, w_init,\
                               phi_slow_init, c1_slow_init, c2_slow_init, w_slow_init, eps)
    loss_slow = loss_func_slow(model_slow, tau_tensor, eps)
    loss_fast2 = loss_func_fast2(model_fast2, t2_tensor, c1_end, c2_end, w_end,\
                                phi_slow_end, c1_slow_end, c2_slow_end, w_slow_end, eps)

    total_loss = weight_fast * loss_fast + weight_slow * loss_slow + weight_fast * loss_fast2
    
    return total_loss

In [ ]:
if __name__=='__main__':

    model_fast  = fast_system_PINN().to(torch.float64)  
    model_slow  = slow_system_PINN().to(torch.float64)  
    model_fast2 = fast_system_PINN2().to(torch.float64) 

    optimizer = torch.optim.Adam(list(model_fast.parameters()) +\
                                 list(model_slow.parameters()) +\
                                 list(model_fast2.parameters()), lr=1e-3)
    loss_values = []
    epoch_num = 50000

    #phi_slow_init, phi_slow_end =  np.random.uniform(V-1, V+1) , np.random.uniform(-0.5, 0.5) 
    #phi_init, phi_end =  np.random.uniform(V-1, V+1) , np.random.uniform(-0.5, 0.5)
    c1_slow_init, c1_slow_end  = np.random.uniform(0, 1) , np.random.uniform(0, 1)
    c2_slow_init, c2_slow_end  = np.random.uniform(0, 1) , np.random.uniform(0, 1)
    J1, J2 = np.random.uniform(0, 1) , np.random.uniform(0, 1)
    for epoch in range(epoch_num):
        optimizer.zero_grad()
        loss_total = total_loss_func(model_fast, model_slow, model_fast2,\
                                    t_tensor, tau_tensor, t2_tensor,\
                                    c1_init, c2_init, w_init,\
                                    phi_slow_init, c1_slow_init, c2_slow_init, w_slow_init,\
                                    c1_end, c2_end, w_end,\
                                    phi_slow_end, c1_slow_end, c2_slow_end, w_slow_end,\
                                    eps, weight_fast=1.0, weight_slow=1.0)
        loss_total.backward()#(retain_graph=True)
        optimizer.step()
        with torch.no_grad():
            phi_pred_fast, u_pred_fast, c1_pred_fast, c2_pred_fast, j1_pred_fast, j2_pred_fast, w_pred_fast = model_fast(t_tensor).numpy().T
            phi_pred_slow, u_pred_slow, c1_pred_slow, c2_pred_slow, j1_pred_slow, j2_pred_slow, w_pred_slow = model_slow(tau_tensor).numpy().T
            phi_pred_fast2, u_pred_fast2, c1_pred_fast2, c2_pred_fast2, j1_pred_fast2, j2_pred_fast2, w_pred_fast2 = model_fast2(t2_tensor).numpy().T
            J1, J2   = j1_pred_slow[-1] , j2_pred_slow[-1]
            #u_a, u_b = u_pred_fast[-1], u_pred_fast2[-1]
            #phi_slow_init = phi_pred_slow[0]
            #phi_slow_end = phi_pred_slow[-1]
            c1_slow_init, c1_slow_end = c1_pred_slow[0], c1_pred_slow[-1]
            c2_slow_init, c2_slow_end = c2_pred_slow[0], c2_pred_slow[-1]

        if epoch % 1000 == 0:
            print(f'Epoch {epoch}, Total Loss: {loss_total.item()}') 
        loss_values.append(loss_total.item())
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 4))
    plt.plot(range(0, epoch_num, 1000), np.log(loss_values[:epoch_num//1000]), 'b', label='Fast System')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.title('Training Loss Over Epochs (Fast System)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Model evaluation
    model_fast.eval()
    model_slow.eval()
    model_fast2.eval()
    with torch.no_grad():
        # Predictions for fast system
        phi_pred_fast, u_pred_fast, c1_pred_fast, c2_pred_fast, j1_pred_fast, j2_pred_fast, w_pred_fast = model_fast(t_tensor).numpy().T
        phi_pred_slow, u_pred_slow, c1_pred_slow, c2_pred_slow, j1_pred_slow, j2_pred_slow, w_pred_slow = model_slow(tau_tensor).numpy().T
        phi_pred_fast2, u_pred_fast2, c1_pred_fast2, c2_pred_fast2, j1_pred_fast2, j2_pred_fast2, w_pred_fast2 = model_fast2(t2_tensor).numpy().T
        #phi_slow_init = phi_pred_slow[0]
        #phi_slow_end = phi_pred_slow[-1]
        c1_slow_init, c1_slow_end = c1_pred_slow[0], c1_pred_slow[-1]
        c2_slow_init, c2_slow_end = c2_pred_slow[0], c2_pred_slow[-1]
        J1, J2   = j1_pred_slow[-1] , j2_pred_slow[-1]
    

Epoch 0, Total Loss: 219.3781848373166
Epoch 1000, Total Loss: 15.32025645005964
Epoch 2000, Total Loss: 0.8155084988436067
Epoch 3000, Total Loss: 0.5820367056159214
Epoch 4000, Total Loss: 0.23257259220353582
Epoch 5000, Total Loss: 0.15407024466487837
Epoch 6000, Total Loss: 0.11590002376383457
Epoch 7000, Total Loss: 0.0871086013912212
Epoch 8000, Total Loss: 0.06868719637773049
Epoch 9000, Total Loss: 0.05418805568764714
Epoch 10000, Total Loss: 0.04479399934993475
Epoch 11000, Total Loss: 0.04093472005426006
Epoch 12000, Total Loss: 0.03310427743432544
Epoch 13000, Total Loss: 0.02871621257426505
Epoch 14000, Total Loss: 0.02622867975348842
Epoch 15000, Total Loss: 0.025615245617460446
Epoch 16000, Total Loss: 0.022891620067805745
Epoch 17000, Total Loss: 0.021748709516722673
Epoch 18000, Total Loss: 0.021964914408567616
Epoch 19000, Total Loss: 0.024358096836673972
Epoch 20000, Total Loss: 0.019399956900067274
Epoch 21000, Total Loss: 0.018886523455372076
Epoch 22000, Total Loss

In [ ]:
#print(J1)
#print(J2)
print(j1_pred_fast.shape)
print(j1_pred_fast[0])
print(j1_pred_fast[-1])
print('')

print(j1_pred_slow[0])
print(j1_pred_slow[-1])
print('')

print(j1_pred_fast2[0])
print(j1_pred_fast2[-1])
print('')

In [ ]:
print(j2_pred_fast.shape)
print(j2_pred_fast[0])
print(j2_pred_fast[-1])
print('')

print(j2_pred_slow[0])
print(j2_pred_slow[-1])
print('')

print(j2_pred_fast2[0])
print(j2_pred_fast2[-1])
print('')

In [ ]:
print(u_pred_fast.shape)
print(u_pred_fast[0])
print(u_pred_fast[-1])
print('')

print(u_pred_slow[0])
print(u_pred_slow[-1])
print('')

print(u_pred_fast2[0])
print(u_pred_fast2[-1])
print('')

u_length = len(u_pred_slow)
print('u_length=',u_length)
zero_vector = np.zeros(u_length)

In [ ]:
print(phi_pred_fast.shape)
print(phi_pred_fast[0])
print(phi_pred_fast[-1])
print(phi_pred_slow[-1])
print('')


In [ ]:
print(z1*c1_pred_fast[0] + z2*c2_pred_fast[0])
print('')
print(z1*c1_pred_fast2[0] + z2*c2_pred_fast2[0])

In [ ]:
print(w_pred_fast.shape)
print(np.max(w_pred_fast))
print(w_pred_fast[-1])
print('')

print(w_pred_slow[0])
print(w_pred_slow[-1])
print('')

print(w_pred_fast2[0])
print(w_pred_fast2[-1])
print('')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('white')  # Set the background color of the 3D plot

# Your existing plots
ax.plot(w_init, u_pred_fast[0], z1 * c1_init + z2 * c2_init, 'go', label='Initial point')
ax.plot(w_pred_fast, u_pred_fast, z1 * c1_pred_fast + z2 * c2_pred_fast, 'b--', label='GSPINN (Fast)')
ax.plot(w_pred_slow + w_pred_fast[-1], u_pred_slow + u_a, z1 * c1_pred_slow + z2 * c2_pred_slow + z1 * c1_slow_init + z2 * c2_slow_init, 'k-', label='GSPINN (Slow)', alpha=0.5)
ax.plot(w_pred_fast2, u_pred_fast2, z1 * c1_pred_fast2 + z2 * c2_pred_fast2, 'm--', label='GSPINN (Fast)', alpha=0.5)
ax.plot(w_end, u_pred_fast2[0], z1 * c1_end + z2 * c2_end, 'ro', label='Ending point')

ax.set_xlabel('$\\mathbf{x}$', fontweight='bold')
ax.set_ylabel('$\\mathbf{u}$', fontweight='bold')
ax.set_zlabel('$\\mathbf{z_1 c_1 + z_2 c_2}$', labelpad=1.8, fontweight='bold')
ax.legend(prop={'weight': 'bold', 'size': 8}, bbox_to_anchor=(0.45, 0.45))
ax.invert_xaxis()

# Set the pane colors to white
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

# Set the grid colors to very light gray
ax.xaxis._axinfo['grid'].update(color=(0.05, 0.05, 0.05, 0.05)) 
ax.yaxis._axinfo['grid'].update(color=(0.05, 0.05, 0.05, 0.05))
ax.zaxis._axinfo['grid'].update(color=(0.05, 0.05, 0.05, 0.05))

# Update tick label properties
ax.tick_params(axis='x', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='y', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='z', labelsize=10, labelcolor='black', width=2, length=5)

# Apply bold font to tick labels
#for label in ax.get_xticklabels() + ax.get_yticklabels() + ax.get_zticklabels():
#    label.set_fontweight('bold')

# Rotating the figure by 20 degrees
ax.view_init(elev=25, azim=120)

plt.grid(True, alpha=0.1)
plt.tight_layout()
plt.show()


In [ ]:
r_0 = z1*c1_pred_fast[0] + z2*c2_pred_fast[0] 
r_1 = z1*c1_pred_fast2[0] + z2*c2_pred_fast2[0] # Initial radius
u_0 = u_pred_fast[0]
u_1 = u_pred_fast2[0]
lambda_ = 0.5 # Rate of shrinking
np_co = 10

# Define the range of t
tt = np.linspace(0, np_co * np.pi, 1000000)

# Calculate r as a function of t
r0 = r_0 * np.exp(-lambda_ * tt)
r1 = r_1 * np.exp(-lambda_ * tt)

u0 = u_0 * np.exp(-lambda_ * tt)
u1 = u_1 * np.exp(-lambda_ * tt)

# Create a similar shrinking track with w = 0
w_exact_f = np.zeros_like(tt)
u_exact_f  = u0 * (np.cos(tt) + np.sin(tt))**2  
zc_exact_f = r0 * (np.cos(tt) + np.sin(tt))**3


# Define the line segment from w = 0 to w = 1 at fixed u = 0, z = 0
w_exact_s = np.linspace(0, 1, 1000)
u_exact_s = np.zeros_like(w_exact_s)
zc_exact_s = np.zeros_like(w_exact_s)

w_exact_f2 = np.ones_like(tt)
u_exact_f2 = u1 * (np.cos(tt) + np.sin(tt))**2
zc_exact_f2 = r1 * (np.cos(tt) + np.sin(tt))**3




In [ ]:
w_exact_fast  = np.zeros_like(w_pred_fast)
u_exact_fast  = np.zeros_like(u_pred_fast)
zc_exact_fast = np.zeros_like(c1_pred_fast)

for i in range(len(w_pred_fast)):
    # Calculate the distance to all points in w_exact_fast, u_exact_fast and z1c1z2c2_exact_fast
    distances = np.sqrt((w_exact_f - w_pred_fast[i])**2 + (u_exact_f - u_pred_fast[i])**2 +\
                        (zc_exact_f - (z1*c1_pred_fast[i]+z2*c2_pred_fast[i]))**2)
    # Find the index of the minimum distance
    closest_index = np.argmin(distances)
    # Assign the closest exact points to x_fast and y_fast
    w_exact_fast[i] = w_exact_f[closest_index]
    u_exact_fast[i] = u_exact_f[closest_index]
    zc_exact_fast[i] = zc_exact_f[closest_index]

In [ ]:
w_exact_slow  = np.zeros_like(w_pred_slow)
u_exact_slow  = np.zeros_like(u_pred_slow)
zc_exact_slow = np.zeros_like(c1_pred_slow)

for i in range(len(w_pred_slow)):
    # Calculate the distance to all points in w_exact_fast, u_exact_fast and z1c1z2c2_exact_fast
    distances2 = np.sqrt((w_exact_s - w_pred_slow[i])**2 + (u_exact_s - u_pred_slow[i])**2 +\
                         (zc_exact_s - (z1*c1_pred_slow[i]+z2*c2_pred_slow[i]))**2)
    # Find the index of the minimum distance
    closest_index2 = np.argmin(distances2)
    # Assign the closest exact points to x_fast and y_fast
    w_exact_slow[i] = w_exact_s[closest_index2]
    u_exact_slow[i] = u_exact_s[closest_index2]
    zc_exact_slow[i] = zc_exact_s[closest_index2]

In [ ]:
w_exact_fast2  = np.zeros_like(w_pred_fast2)
u_exact_fast2  = np.zeros_like(u_pred_fast2)
zc_exact_fast2 = np.zeros_like(c1_pred_fast2)

for i in range(len(w_pred_fast2)):
    # Calculate the distance to all points in w_exact_fast, u_exact_fast and z1c1z2c2_exact_fast
    distances3 = np.sqrt((w_exact_f2 - w_pred_fast2[i])**2 + (u_exact_f2 - u_pred_fast2[i])**2 +\
                        (zc_exact_f2 - (z1*c1_pred_fast2[i]+z2*c2_pred_fast2[i]))**2)
    # Find the index of the minimum distance
    closest_index3 = np.argmin(distances3)
    # Assign the closest exact points to x_fast and y_fast
    w_exact_fast2[i] = w_exact_f2[closest_index3]
    u_exact_fast2[i] = u_exact_f2[closest_index3]
    zc_exact_fast2[i] = zc_exact_f2[closest_index3]

In [ ]:

# Create the 3D plot again with the new track
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('white') 


# Plot the similar track starting with w = 0
ax.plot(w_exact_fast, u_exact_fast, zc_exact_fast, color='orange', marker='.', linestyle='None')



# Plot the line segment
ax.plot(w_exact_slow, u_exact_slow, zc_exact_slow,  color='orange', linestyle='-')

# Plot the original shrinking track
ax.plot(w_exact_fast2, u_exact_fast2, zc_exact_fast2,  color='orange', marker='.', linestyle='None')

# Set the grid colors to very light gray
ax.xaxis._axinfo['grid'].update(color=(0.05, 0.05, 0.05, 0.05)) 
ax.yaxis._axinfo['grid'].update(color=(0.05, 0.05, 0.05, 0.05))
ax.zaxis._axinfo['grid'].update(color=(0.05, 0.05, 0.05, 0.05))

# Update tick label properties
ax.tick_params(axis='x', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='y', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='z', labelsize=10, labelcolor='black', width=2, length=5)


# Set labels and title
ax.set_xlabel('w')
ax.set_ylabel('u')
ax.set_zlabel('$z_1c_1 + z_2c_2$')
ax.zaxis.labelpad = -1
ax.legend()
#ax.invert_xaxis()  
ax.invert_yaxis()
ax.view_init(elev=20)

# Show the plot
plt.show()


In [ ]:
# Calculate the error
w_error_fast = np.abs(w_exact_fast - w_pred_fast)
u_error_fast = np.abs(u_exact_fast - u_pred_fast)
zc_error_fast = np.abs(zc_exact_fast - (z1*c1_pred_fast+z2*c2_pred_fast))


w_error_slow = np.abs(w_exact_slow - w_pred_slow)
u_error_slow = np.abs(u_exact_slow - u_pred_slow)
zc_error_slow = np.abs(zc_exact_slow - (z1*c1_pred_slow+z2*c2_pred_slow))


w_error_fast2 = np.abs(w_exact_fast2 - w_pred_fast2)
u_error_fast2 = np.abs(u_exact_fast2 - u_pred_fast2)
zc_error_fast2 = np.abs(zc_exact_fast2 - (z1*c1_pred_fast2+z2*c2_pred_fast2))



# Maximum error
max_w_error_f = np.max(w_error_fast)
max_u_error_f = np.max(u_error_fast)
max_zc_error_f = np.max(zc_error_fast)


print("Maximum errors for the first piece (Fast layer):")
print("Max w error fast:", max_w_error_f)
print("Max u error fast:", max_u_error_f)
print("Max zc error fast:", max_zc_error_f)




# Maximum error for the slow layer
max_w_error_s = np.max(w_error_slow)
max_u_error_s = np.max(u_error_slow)
max_zc_error_s = np.max(zc_error_slow)


print("Maximum errors over the critical manifold:")
print("Max w error slow:", max_w_error_s)
print("Max u error slow:", max_u_error_s)
print("Max zc error slow:", max_zc_error_s)



# Maximum error for the third piece
max_w_error_f2 = np.max(w_error_fast2)
max_u_error_f2 = np.max(u_error_fast2)
max_zc_error_f2 = np.max(zc_error_fast2)


print("Maximum errors for the first piece (Fast layer):")
print("Max w error fast2:", max_w_error_f2)
print("Max u error fast2:", max_u_error_f2)
print("Max zc error fast2:", max_zc_error_f2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data (replace these with actual errors)
tt_end = 1
time = np.linspace(0, tt_end, 100)

# Plot the errors horizontally (1 row, 3 columns)
plt.figure(figsize=(18, 6))

# Plot 1: Fast errors
plt.subplot(1, 3, 1)
plt.plot(time, w_error_fast, label='$w$ error', color='b', linestyle='-', alpha=0.7)
plt.plot(time, u_error_fast, label='$u$ error', color='r', linestyle='--', alpha=0.7)
plt.plot(time, zc_error_fast, label='$z_1c_1+z_2c_2$ error', color='g', linestyle='-.', alpha=0.7)
plt.title('Errors for Fast Trajectory',fontweight='bold')
plt.xlabel('Time',fontweight='bold')
plt.ylabel('Error',fontweight='bold')
plt.legend()
plt.grid()

# Plot 2: Slow errors
plt.subplot(1, 3, 2)
plt.plot(time, w_error_slow, label='$w$ error', color='b', linestyle='-', alpha=0.7)
plt.plot(time, u_error_slow, label='$u$ error', color='r', linestyle='--', alpha=0.7)
plt.plot(time, zc_error_slow, label='$z_1c_1+z_2c_2$ error', color='g', linestyle='-.', alpha=0.7)
plt.title('Errors for Slow Trajectory', fontweight='bold')
plt.xlabel('Time', fontweight='bold')
plt.ylabel('Error', fontweight='bold' )
plt.legend()
plt.grid()

# Plot 3: Fast2 errors
plt.subplot(1, 3, 3)
plt.plot(time, w_error_fast2, label='$w$ error', color='b', linestyle='-', alpha=0.7)
plt.plot(time, u_error_fast2, label='$u$ error', color='r', linestyle='--', alpha=0.7)
plt.plot(time, zc_error_fast2, label='$z_1c_1+z_2c_2$ error', color='g', linestyle='-.', alpha=0.7)
plt.title('Errors for Fast2 Trajectory', fontweight='bold')
plt.xlabel('Time',fontweight='bold')
plt.ylabel('Error',fontweight='bold')
plt.legend()
plt.grid()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(tau, phi_pred_slow, 'b--', label='predicted $\\mathbf{\\phi}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, c1_pred_slow, 'g--', label='predicted $\\mathbf{c_1}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(tau, c2_pred_slow, 'm-.', label='predicted $\\mathbf{c_2}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6.4, 4.8))

# Plot the data
plt.plot(tau, j1_pred_slow, 'b--', label='Predicted $\\mathbf{J_1}$')
plt.plot(tau, j2_pred_slow, '-.', color='orange', label='Predicted $\\mathbf{J_2}$')
plt.plot(tau, z1*j1_pred_slow + z2*j2_pred_slow, '-', color='green', label='Predicted $\\mathbf{I}$')

# Label the axes
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.ylabel('$\\mathbf{Flux (J)}$', fontweight='bold')

plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

# Add grid and legend
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, j1_pred_fast, 'b--', label='Predicted $\\mathbf{J_1}$')
plt.plot(t, j2_pred_fast, '-.', color='orange', label='Predicted $\\mathbf{J_2}$')
plt.plot(t, z1*j1_pred_fast + z2*j2_pred_fast, '-', color='green', label='Predicted $\\mathbf{I}$')

plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title('Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, j1_pred_slow, 'b--', label='Predicted $\\mathbf{J_1}$')
plt.plot(tau, j2_pred_slow, '-.', color='orange', label='Predicted $\\mathbf{J_2}$')
plt.plot(tau, z1*j1_pred_slow + z2*j2_pred_slow, '-', color='green', label='Predicted $\\mathbf{I}$')

plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.title('Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, j1_pred_fast2 , 'b--', label='Predicted $\\mathbf{J_1}$')
plt.plot(t2, j2_pred_fast2 , '-.', color='orange', label='Predicted $\\mathbf{J_2}$')
plt.plot(t2, z1*j1_pred_fast2 + z2*j2_pred_fast2, '-', color='green', label='Predicted $\\mathbf{I}$')

plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title('Over the 3rd piece (fast layer)', fontweight='bold')
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.grid(True, alpha=0.1)
plt.tight_layout()
plt.show()
j2_pred_fast[:2]

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, z1 * c1_pred_fast + z2 * c2_pred_fast, 'm--', label='Predicted $\\mathbf{z_1 c_1+ z_2  c_2}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title(' Over the first piece  (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

zc_slow_init = z1 * c1_pred_fast[-1] + z2 * c2_pred_fast[-1]
plt.subplot(1, 3, 2)
plt.plot(tau, z1 * c1_pred_slow + z2 * c2_pred_slow + zc_slow_init, 'm--', label='Predicted $\\mathbf{z_1 c_1+ z_2  c_2}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.ylim(-0.1, 0.1)
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, z1 * c1_pred_fast2 + z2 * c2_pred_fast2, 'm--', label='Predicted $\\mathbf{z_1 c_1+ z_2  c_2}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title(' Over the 3rd piece  (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.tight_layout()
plt.show()


### In the following, we would like to investigate whether the variables are fast or slow based on their dynamic on fast layers and slow layer:

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, phi_pred_fast, 'k-', label='Predicted $\\mathbf{\\phi}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{\\phi}$', fontweight='bold')
plt.title(' Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, phi_pred_slow, 'k-', label='Predicted $\\mathbf{\\phi}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.ylabel('$\\mathbf{\\phi}$', fontweight='bold')
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, phi_pred_fast2 , 'k-', label='Predicted $\\mathbf{\\phi}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{\\phi}$', fontweight='bold')
plt.title(' Over the 3rd piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, u_pred_fast, 'r-.', label='Predicted $\\mathbf{u}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{u}$', fontweight='bold')
plt.title(' Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, u_pred_slow, 'r-.', label='Predicted $\\mathbf{u}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.ylabel('$\\mathbf{u}$', fontweight='bold')
plt.ylim(-0.1, 0.1)
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, u_pred_fast2, 'r-.', label='Predicted $\\mathbf{u}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{u}$', fontweight='bold')
plt.title(' Over the 3rd piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, c1_pred_fast, 'g--', label='Predicted $\\mathbf{c_1}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{c_1}$', fontweight='bold')
plt.title(' Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, c1_pred_slow, 'g--', label='Predicted $\\mathbf{c_1}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.ylabel('$\\mathbf{c_1}$', fontweight='bold')
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, c1_pred_fast2, 'g--', label='Predicted $\\mathbf{c_1}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{c_1}$', fontweight='bold')
plt.title(' Over the 3rd piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, c2_pred_fast, 'm-.', label='Predicted $\\mathbf{c_2}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{c_2}$', fontweight='bold')
plt.title(' Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, c2_pred_slow, 'm-.', label='Predicted $\\mathbf{c_2}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.ylabel('$\\mathbf{c_2}$', fontweight='bold')
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, c2_pred_fast2, 'm-.', label='Predicted $\\mathbf{c_2}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.ylabel('$\\mathbf{c_2}$', fontweight='bold')
plt.title(' Over the 3rd piece (fast layer)', fontweight='bold')
plt.legend(prop={'weight': 'bold'})

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.grid(True, alpha=0.1)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

t_end = 20
t   =  np.linspace(0, t_end, 100)
t2  =  np.linspace(0,-t_end, 100)

plt.subplot(1, 3, 1)
plt.plot(t, j1_pred_fast, 'b-', label='Predicted $\\mathbf{J_1}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title(' Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})
plt.ylim(-1.0, 0)

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, j1_pred_slow, 'b-', label='Predicted $\\mathbf{J_1}$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})
plt.ylim(-1.0, 0)

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, j1_pred_fast2, 'b-', label='Predicted $\\mathbf{J_1}$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title(' Over the 3rd piece (fast layer)', fontweight='bold')
plt.legend(prop={'weight': 'bold'})
plt.ylim(-1.0, 0)

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.grid(True, alpha=0.1)
plt.tight_layout()
plt.show()

j1_pred_fast[:5]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(t, j2_pred_fast, '-.', color='orange', label='Predicted $J_2$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title(' Over the first piece (fast layer)', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})
plt.ylim(0.0, 1.0)

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 2)
plt.plot(tau, j2_pred_slow, '-.', color='orange', label='Predicted $J_2$')
plt.xlabel('$\\mathbf{x}$', fontweight='bold')
plt.title(' Over the critical manifold', fontweight='bold')
plt.grid(True, alpha=0.1)
plt.legend(prop={'weight': 'bold'})
plt.ylim(0.0, 1.0)

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.subplot(1, 3, 3)
plt.plot(t2, j2_pred_fast2, '-.', color='orange', label='Predicted $J_2$')
plt.xlabel('$\\mathbf{\\xi}$', fontweight='bold')
plt.title(' Over the 3rd piece (fast layer)', fontweight='bold')
plt.legend(prop={'weight': 'bold'})
plt.ylim(0.0, 1.0)

# Making tick labels bold
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

plt.grid(True, alpha=0.1)
plt.tight_layout()
plt.show()

j2_pred_fast2[:5]

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


eps =0.01
z1 = 1
z2= -1

def fast_system(y, t, eps):
    u, c1, c2, w = y
    dudt   = -z1 * c1 - z2 * c2
    dc1dt  = -z1 * c1 * u #- eps * j1
    dc2dt  = -z2 * c2 * u #- eps * j2
    dwdt   = 0 #eps
    
    return [dudt, dc1dt, dc2dt, dwdt]

# Initial conditions
z1= 1
z2 = -1
u0 = u_pred_fast[0]
l1 = c1_init
l2 = c2_init
y0 = [ u0, l1, l2, 0]  # Initial values [u, c1, c2, w]


# Time vector
t_end = 3
t2_1 = np.linspace(0, t_end, 10)


# Solve the system
solution = odeint(fast_system, y0, t2_1, args=(eps,))
# Extract results
u, c1, c2, w = solution.T


# Plot the 3D path
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(w_init, u0,  z1 * l1 + z2 * l2, color='g', label='Initial Point ', s=30)
ax.plot(w, u, z1 * c1 + z2 * c2, '-',  color='g', label='ODEINT')
ax.plot(w_pred_fast, u_pred_fast, z1 * c1_pred_fast + z2 * c2_pred_fast, 'b--', label='GSPINN (Fast)')
ax.plot(w_pred_slow, u_pred_slow, z1 * c1_pred_slow + z2 * c2_pred_slow, 'k-', alpha=0.5)
ax.scatter(w_pred_fast[-1],u_a, z1 * c1_slow_init + z2 * c2_slow_init, color='orange', label='(0, 0, 0)', s=30)


ax.set_xlabel('$\\mathbf{x}$', fontweight='bold')
ax.set_ylabel('$\\mathbf{u}$', fontweight='bold')
ax.set_zlabel('$\\mathbf{z_1 c_1 + z_2 c_2}$', labelpad=1, fontweight='bold')
ax.invert_xaxis()

# Set the pane colors to white
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

# Set the grid colors to very light gray
ax.xaxis._axinfo['grid'].update(color=(0.03, 0.03, 0.03, 0.03)) 
ax.yaxis._axinfo['grid'].update(color=(0.03, 0.03, 0.03, 0.03))
ax.zaxis._axinfo['grid'].update(color=(0.03, 0.03, 0.03, 0.03))

# Update tick label properties
ax.tick_params(axis='x', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='y', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='z', labelsize=10, labelcolor='black', width=2, length=5)

# Apply bold font to tick labels
#for label in ax.get_xticklabels() + ax.get_yticklabels() + ax.get_zticklabels():
#    label.set_fontweight('bold')

# Labels and legend
ax.legend(prop={'weight': 'bold', 'size': 8}, bbox_to_anchor=(0.5, 0.6))
ax.set_title('Over the fast layer')
ax.view_init(elev=20, azim=115)

plt.grid(True, alpha=0.1)
plt.show()


In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


eps =0.01
z1 = 1
z2= -1

def fast_system(y, t, eps):
    u, c1, c2, w = y
    dudt   = -z1 * c1 - z2 * c2
    dc1dt  = -z1 * c1 * u #- eps * j1
    dc2dt  = -z2 * c2 * u #- eps * j2
    dwdt   = 0 #eps
    
    return [dudt, dc1dt, dc2dt, dwdt]

# Initial conditions
z1= 1
z2 = -1
u0 = u_pred_fast[0]
u1 = u_pred_fast2[0]
l1 = c1_init
l2 = c2_init
r1 = c1_pred_fast2[0]
r2 = c2_pred_fast2[0]
y0 = [ u0, l1, l2, 0]  # Initial values [u, c1, c2, w]
y1 = [ u1, r1, r2, 1] 

# Time vector
t_end_1 = 3
t_end_2 = 3
t2_1 = np.linspace(0, t_end_1, 10)
t2_2 = np.linspace(0, -t_end_2, 10)

# Solve the system
solution = odeint(fast_system, y0, t2_1, args=(eps,))
solution2 = odeint(fast_system, y1, t2_2, args=(eps,))
# Extract results
u, c1, c2, w = solution.T
u_2, c1_2, c2_2, w_2 = solution2.T

# Plot the 3D path
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(w_init, u0,  z1 * l1 + z2 * l2, color='g', label='Initial Point ', s=30)
ax.plot(w, u, z1 * c1 + z2 * c2, '-',  color='g', label='ODEINT')
#ax.plot(w_init, u_pred_fast[0], z1 * c1_init + z2 * c2_init, 'go', label='Initial point')
ax.plot(w_pred_fast, u_pred_fast, z1 * c1_pred_fast + z2 * c2_pred_fast, 'b--', label='GSPINN (Fast)')
ax.plot(w_pred_slow, u_pred_slow, z1 * c1_pred_slow + z2 * c2_pred_slow, 'k-', alpha=0.5)
ax.scatter(w_pred_fast[-1],u_a, z1 * c1_slow_init + z2 * c2_slow_init, color='orange', label='(0, 0, 0)', s=30)
ax.plot(w_pred_fast2, u_pred_fast2, z1 * c1_pred_fast2 + z2 * c2_pred_fast2, 'm--', label='GSPINN (Fast)', alpha=0.5)
ax.plot(w_2, u_2, z1 * c1_2 + z2 * c2_2, '-',  color='g', label='ODEINT')
ax.plot(w_end, u_pred_fast2[0], z1 * c1_end + z2 * c2_end, 'ro', label='Ending point')


ax.set_xlabel('$\\mathbf{x}$', fontweight='bold')
ax.set_ylabel('$\\mathbf{u}$', fontweight='bold')
ax.set_zlabel('$\\mathbf{z_1 c_1 + z_2 c_2}$', labelpad=1, fontweight='bold')
ax.invert_xaxis()

# Set the pane colors to white
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

# Set the grid colors to very light gray
ax.xaxis._axinfo['grid'].update(color=(0.03, 0.03, 0.03, 0.03)) 
ax.yaxis._axinfo['grid'].update(color=(0.03, 0.03, 0.03, 0.03))
ax.zaxis._axinfo['grid'].update(color=(0.03, 0.03, 0.03, 0.03))

# Update tick label properties
ax.tick_params(axis='x', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='y', labelsize=10, labelcolor='black', width=2, length=5)
ax.tick_params(axis='z', labelsize=10, labelcolor='black', width=2, length=5)

# Apply bold font to tick labels
for label in ax.get_xticklabels() + ax.get_yticklabels() + ax.get_zticklabels():
    label.set_fontweight('bold')

# Labels and legend
ax.legend(prop={'weight': 'bold', 'size': 6}, bbox_to_anchor=(0.55, 0.5))
ax.view_init(elev=20, azim=115)

plt.grid(True, alpha=0.1)
plt.show()
